<a href="https://colab.research.google.com/github/ankalagigaurave/ML_SEM_V/blob/master/Labs/ML_Lab_12_J004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
dfc = pd.read_csv("airqual.csv")
dfc.isnull().sum()

In [ ]:
dfc.info()

In [ ]:
dfc.describe()

In [ ]:
dfc.shape

In [ ]:
dfc.replace(to_replace = -200,value = np.NaN , inplace = True )

In [ ]:
num = dfc.isnull().sum()
percent = num/dfc.shape[0]*100
percent

In [ ]:
dfc['DateTime'] = dfc['Date']+ ' ' + dfc['Time']
dfc.drop(['Time'], axis=1, inplace=True)
dfc['DateTime'] = pd.to_datetime(dfc['DateTime'].astype(str), format='%d-%m-%Y %H:%M:%S')
dfc['Date'] = pd.to_datetime(dfc['Date'].astype(str), format='%d-%m-%Y')
dfc.shape

In [ ]:
dfc.head()

In [ ]:
num = dfc.isnull().sum()
percent = num/dfc.shape[0]*100
percent

In [ ]:
dfc.drop(['NMHC(GT)'], axis= 1, inplace= True)
num = dfc.isnull().sum()
percent = num/dfc.shape[0]*100
percent

In [ ]:
num

In [ ]:
dfc["T"] = dfc.groupby("Date")["T"].transform(lambda x: x.fillna(x.mean()))
dfc["CO(GT)"] = dfc.groupby("Date")["CO(GT)"].transform(lambda x: x.fillna(x.mean()))
dfc["NOx(GT)"] = dfc.groupby("Date")["NOx(GT)"].transform(lambda x: x.fillna(x.mean()))
dfc["NO2(GT)"] = dfc.groupby("Date")["NO2(GT)"].transform(lambda x: x.fillna(x.mean()))

In [ ]:
num = dfc.isnull().sum()
percent = num/dfc.shape[0]*100
percent

In [ ]:
sns.heatmap(dfc.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
dfc.fillna(method='ffill', inplace= True)

In [ ]:
num = dfc.isnull().sum()
percent = num/dfc.shape[0]*100
percent

In [ ]:
dfc.info()

In [ ]:
dfc.describe()

In [ ]:
dfc.plot(x='Date',subplots=True, figsize=(15, 10),legend=False)

In [ ]:
plt.figure(); dfc.plot(x='Date',y='CO(GT)',figsize=(15, 10)); plt.legend(loc='best')

In [ ]:
sns.distplot(dfc["CO(GT)"])

In [ ]:
sns.distplot(dfc["NOx(GT)"])

In [ ]:
sns.distplot(dfc["NO2(GT)"])

In [ ]:
sns.distplot(dfc["C6H6(GT)"])

In [ ]:
sns.distplot(dfc["T"])

In [ ]:
sns.distplot(dfc["RH"])

In [ ]:
sns.distplot(dfc["AH"])

In [ ]:
sns.heatmap(dfc.corr(), annot=True)

In [ ]:
dfc.head()

In [ ]:
X = dfc[['CO(GT)','PT08.S1(CO)','C6H6(GT)','PT08.S2(NMHC)','NOx(GT)'	,'PT08.S3(NOx)','NO2(GT)','PT08.S4(NO2)','PT08.S5(O3)','T','RH']]

In [ ]:
Y = dfc[['AH']]

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
dfc.shape

In [ ]:
n = len(dfc)
x_train_df = X[0:int(n*0.8)]
x_test_df = X[int(n*0.8):]
y_train_df = Y[0:int(n*0.8)]
y_test_df = Y[int(n*0.8):]

In [ ]:
print(x_train_df.shape,y_train_df.shape,x_test_df.shape,y_test_df.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train_df)
x_train = scaler.fit_transform(x_train_df)
scaler.fit(x_test_df)
x_test = scaler.fit_transform(x_test_df)
scaler.fit(y_train_df)
y_train = scaler.fit_transform(y_train_df)
scaler.fit(y_test_df)
y_test = scaler.fit_transform(y_test_df)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
def RNN_MODEL():
    # have to convert to 3D for feeding the data
    regressor = Sequential()
    regressor.add(LSTM(units = 50,return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50,return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = 1))
    regressor.compile(optimizer = 'adam',loss = 'mean_squared_error',metrics=['accuracy'])
    return regressor
regressor = RNN_MODEL()
regressor.fit(x_train,y_train,epochs = 100)

In [ ]:
regressor.evaluate(x_test,y_test)
y_pred = regressor.predict(x_test)
output = scaler.inverse_transform(y_pred)
real_output = []
for item in output:
    real_output.append((item[0]))

In [ ]:
value = y_test_df
value['predicted'] = np.array(real_output)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)
value.plot()